In [ ]:
!pip install pygad
!pip install optuna

In [1]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import pygad
import optuna
from statistics import median,mean
import random
from multiprocessing import Pool

In [23]:
base_As=[np.random.randint(low=0,high=2,size=(random.randint(5,10),random.randint(5,10))) for i in range(25)]

In [24]:
def obj(trial):
    params = {
        "sol_per_pop": trial.suggest_int("sol_per_pop", 50, 100),
        "num_parents_mating":2,
        "crossover_probability":trial.suggest_float("crossover_probability",0.8,0.9,log=False),
        "mutation_probability":trial.suggest_float("mutation_probability",0.01,0.05,log=False),
        "gene_space":[0,1],
        "parent_selection_type":trial.suggest_categorical("parent_selection_type", ["rws","tournament"]),
        "num_generations":100
    }
    num_iters=[]
    fit_vals=[]

    for A in base_As:
       params["num_genes"]=len(A[0])
       params["stop_criteria"]=f"reach_{len(A[0])}"
       params["fitness_func"]= lambda solution, solution_idx: sum([1 if item[0]==1 else 0 for item in np.matmul(A,np.array([[ii] for ii in solution]))])
       ga_instance = pygad.GA(**params)
       ga_instance.run()
       solution, solution_fitness, solution_idx = ga_instance.best_solution()
       num_iters.append(ga_instance.best_solution_generation)
       fit_vals.append(solution_fitness)
    
    niter=mean(num_iters)
    fval=mean(fit_vals)
    return niter,fval
study = optuna.create_study(directions=["minimize", "maximize"])
study.optimize(obj, n_trials=30)#, timeout=300

print("Number of finished trials: ", len(study.trials))

[I 2023-04-04 18:49:34,143] A new study created in memory with name: no-name-37fd2c0b-75f7-47dd-810c-cd84021d22c0
[I 2023-04-04 18:49:41,756] Trial 0 finished with values: [10.0, 6.0] and parameters: {'sol_per_pop': 84, 'crossover_probability': 0.8688986318531717, 'mutation_probability': 0.03345344809207382, 'parent_selection_type': 'rws'}. 
[I 2023-04-04 18:49:47,083] Trial 1 finished with values: [10.0, 6.0] and parameters: {'sol_per_pop': 88, 'crossover_probability': 0.8011676236855945, 'mutation_probability': 0.018154946548950016, 'parent_selection_type': 'tournament'}. 
[I 2023-04-04 18:49:52,727] Trial 2 finished with values: [7.0, 6.0] and parameters: {'sol_per_pop': 81, 'crossover_probability': 0.8927765503661904, 'mutation_probability': 0.028734914770389454, 'parent_selection_type': 'rws'}. 
[I 2023-04-04 18:49:57,501] Trial 3 finished with values: [14.0, 6.0] and parameters: {'sol_per_pop': 54, 'crossover_probability': 0.868413893372346, 'mutation_probability': 0.047373153739

Number of finished trials:  30


In [25]:
optuna.visualization.plot_parallel_coordinate(study,target=lambda t: t.values[1], target_name="fval")

In [26]:
optuna.visualization.plot_param_importances(study, target=lambda t: t.values[0], target_name="niter")